In [ ]:
# Human specific
import inspect

# This prompt is ignored if the environment_name already corresponds to an existing environment.
specific = inspect.cleandoc("""\
                            You plan to visit 3 European cities for 14 days in total. 
                            You only take direct flights to commute between cities. 
                            You would like to visit Florence for 6 days. 
                            You want to meet a friend in Florence between day 9 and day 14. 
                            You would like to visit Barcelona for 5 days. 
                            You would like to visit Helsinki for 5 days.
                            
                            Here are the cities that have direct flights:
                            Barcelona and Florence, Helsinki and Barcelona.
                            
                            Find a trip plan of visiting the cities for 14 days by taking direct flights to commute between them.
                            """)
environment_name = "TripPlanning"
format = "json"

In [ ]:
# Generate the json representation of the environment
from pathlib import Path

from src.llm_plan.planner import Planner
from src.llm_plan.llm import ChatGPT
from src.llm_plan.config import ENVIRONMENTS_JSON_PATH

model = ChatGPT("gpt-5-mini")  # This is one of the many models you can use now
planner = Planner()

plan_path = Path(f"{environment_name}.{format}")
full_path = ENVIRONMENTS_JSON_PATH / plan_path

# Skip if the plan already exists
if not full_path.exists():
    planner.generate_representation(model, specific, environment_name, format=format)
else:
    print(f"{full_path} already exists. Skipping generation.")
    print("[Warning]: The prompt `specific` will be ignored!")

In [ ]:
# Generate the environment plan
from src.llm_plan.environment import Environment

env = Environment(f"./environments/static/{environment_name}.json")
print("Plan:\n", env.plan)

In [19]:
# Plan
model = ChatGPT("gpt-4o")  # switch to a less powerful model for planning
responses = planner.plan(model, env)

----- travelerA->pddl -----

----- System Prompt -----
You are an expert with PDDL problems Planning Domain Definition Language. You always provide a PDDL domain and a PDDL problem file to solve the task. You always enclose the pddl domain between <domain></domain> tags and the pddl problem between <problem></problem> tags.

----- Prompt -----
Your name is travelerA. You are in an environment with the following public information ['You plan to visit 3 European cities for 14 days in total', 'You only take direct flights to commute between cities', 'The direct flights available are Barcelona and Florence, and Helsinki and Barcelona']. You have the following knowledge ['I would like to visit Florence for 6 days', 'I want to meet a friend in Florence between day 9 and day 14']. This is the global goal to solve Find a trip plan of visiting the cities for 14 days by taking direct flights to commute between them. Think step by step and provide a PDDL domain and a PDDL problem file to solve th

In [20]:
# Extract the pddl problem and domain from the LLM response
from src.llm_plan.parser import PDDLParser

pddl_parser = PDDLParser()

# TODO: fix the vocabulary entry, it shouldn't be "pddl_scheduler" etc.
final_plan = responses["pddl_orchestrator"]
domain, problem = pddl_parser.parse(final_plan, from_file=False)

In [21]:
# Obtain the plan with the solver
import subprocess

from pathlib import Path

from src.llm_plan.config import SOLVER_BINARY, SOLVER_ARGS

BASE_FOLDER = Path(f"./tmp/{env.name}")
BASE_FOLDER.mkdir(parents=True, exist_ok=True)

with open(BASE_FOLDER / "problem.pddl", "w") as f:
    f.write(problem)

with open(BASE_FOLDER / "domain.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    BASE_FOLDER / "sas_plan",
    BASE_FOLDER / "domain.pddl",
    BASE_FOLDER / "problem.pddl",
]

with open(BASE_FOLDER / "logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)

In [22]:
# Validate the plan with uVAL
import subprocess

from src.llm_plan.config import UNIVERSAL_VALIDATOR_BIN

command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
{BASE_FOLDER / 'domain.pddl'} \
{BASE_FOLDER / 'problem.pddl'} \
{BASE_FOLDER / 'sas_plan'}"

out = subprocess.run(command, shell=True, capture_output=True, text=True)

# This part won't be printed at test time
if out.stderr:
    pddl_error = out.stderr
    print(
        f"The validation found a problem with the plan: {out.stderr}"
    )
else:
    pddl_error = "No error found."
    print("The plan is valid.")
    print("[stdout]", out.stdout)

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type std::runtime_error: Failed to open file 'tmp/EuroTripDirectFlights/sas_plan'



In [23]:
# Let's try to refine the plan syntax
from src.llm_plan.hypervisor import (
    HypervisorDeepThinkPDDL,
    HypervisorSyntaxPDDL,
    HypervisorFastDownwardAdapter
    )

num_attempts = 10

for i in range(num_attempts):
    
    # --- General validator of PDDL plans ---
    prompt_args = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
    }

    deep_think = HypervisorDeepThinkPDDL(llm=model, prompt_args=prompt_args)

    r = deep_think.run()
    domain, problem = pddl_parser.parse(r, from_file=False)
    
    # --- Fast Downward adapter ---
    prompt_args = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
    }
    
    hypervisor_fd = HypervisorFastDownwardAdapter(llm=model, prompt_args=prompt_args)
    
    r = hypervisor_fd.run()
    domain, problem = pddl_parser.parse(r, from_file=False)
    
    
    # Obtain the new plan with the solver
    with open(BASE_FOLDER / f"refined_problem_{i}.pddl", "w") as f:
        f.write(problem)

    with open(BASE_FOLDER / f"refined_domain_{i}.pddl", "w") as f:
        f.write(domain)
        
    # Launch the solver
    command = [
        SOLVER_BINARY,
        *SOLVER_ARGS,
        BASE_FOLDER / f"refined_sas_plan_{i}",
        BASE_FOLDER / f"refined_domain_{i}.pddl",
        BASE_FOLDER / f"refined_problem_{i}.pddl",
    ]

    with open(BASE_FOLDER / f"refined_logs_{i}.txt", "w") as logfile:
        subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)
        
    command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
    {BASE_FOLDER / f'refined_domain_{i}.pddl'} \
    {BASE_FOLDER / f'refined_problem_{i}.pddl'} \
    {BASE_FOLDER / f'refined_sas_plan_{i}'}"

    out = subprocess.run(command, shell=True, capture_output=True, text=True)

    # This part won't be printed at test time
    if out.stderr:
        pddl_error = out.stderr
        print(
            f"The validation found a problem with the plan: {out.stderr}"
        )
    else:
        pddl_error = "No error was found."
        print("The plan is valid.")
        print("[stdout]", out.stdout)
    
    # --- Syntax validator ---
    print(f"Validation attempt {i+1}/{num_attempts}")
    prompt_args = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
        "syntax_errors": pddl_error,
    }

    hypervisor_pddl = HypervisorSyntaxPDDL(llm=model, prompt_args=prompt_args)

    r = hypervisor_pddl.run()
    domain, problem = pddl_parser.parse(r, from_file=False)

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: DAYS-PLANNED does not name a known token

Validation attempt 1/10
The plan is valid.
[stdout] Checking plan: tmp/EuroTripDirectFlights/refined_sas_plan_1
Plan executed successfully - checking goal
Success: Plan valid
Value: 14.000000


Validation attempt 2/10
The plan is valid.
[stdout] Checking plan: tmp/EuroTripDirectFlights/refined_sas_plan_2
Plan executed successfully - checking goal
Success: Plan valid
Value: 13.000000


Validation attempt 3/10
The plan is valid.
[stdout] Checking plan: tmp/EuroTripDirectFlights/refined_sas_plan_3
Plan executed successfully - checking goal
Success: Plan valid
Value: 12.000000


Validation attempt 4/10
The plan is valid.
[stdout] Checking plan: tmp/EuroTripDirectFlights/refined_sas_plan_4
Plan executed successfully - checking goal
Success: Plan valid
Value: 34.000000


Validation attempt 5/10
The plan is valid.
[stdout]

In [24]:
# Final plan run and validation
with open(BASE_FOLDER / f"refined_problem_{i}.pddl", "w") as f:
    f.write(problem)

with open(BASE_FOLDER / f"refined_domain_{i}.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    BASE_FOLDER / f"refined_sas_plan_{i}",
    BASE_FOLDER / f"refined_domain_{i}.pddl",
    BASE_FOLDER / f"refined_problem_{i}.pddl",
]

with open(BASE_FOLDER / "refined_logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)
    
command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
{BASE_FOLDER / f'refined_domain_{i}.pddl'} \
{BASE_FOLDER / f'refined_problem_{i}.pddl'} \
{BASE_FOLDER / f'refined_sas_plan_{i}'}"

out = subprocess.run(command, shell=True, capture_output=True, text=True)

# This part won't be printed at test time
if out.stderr:
    pddl_error = out.stderr
    print(
        f"The validation found a problem with the plan: {out.stderr}"
    )
else:
    pddl_error = "No error found."
    print("The plan is valid.")
    print("[stdout]", out.stdout)

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type std::runtime_error: Failed to open file 'tmp/EuroTripDirectFlights/refined_sas_plan_9'



In [26]:
# Report the natural language plan
from src.llm_plan.hypervisor import HypervisorNaturalLanguage

with open(BASE_FOLDER / f"refined_problem_{i}.pddl", "r") as f:
    problem = f.read()

with open(BASE_FOLDER / f"refined_domain_{i}.pddl", "r") as f:
    domain = f.read()
    
with open(BASE_FOLDER / f"refined_sas_plan_{i}", "r") as f:
    plan = f.read()
    
prompt_args = {
    "specification": env.config_data,
    "pddl_domain": domain,
    "pddl_problem": problem,
    "pddl_plan": plan,
}

hypervisor_to_nl = HypervisorNaturalLanguage(llm=model, prompt_args=prompt_args)

natural_plan = hypervisor_to_nl.run()

with open(BASE_FOLDER / "refined_nl_plan.txt", "w") as f:
    f.write(natural_plan)